In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SparkContext
from lib.prepro import Preprocessing, SparkDataFrame
from tqdm.notebook import tqdm
import dask.dataframe as dd

In [2]:
PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터/*'
PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/*'
PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터/*'

# PATH_CCTV_DATA = './cctv_data/*'
# PATH_SAND_DATA = './sand_data/*'
# PATH_SAND_LABEL = './sand_label_data/*'

# PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터'
# PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터'
# PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터'

In [3]:
prepro = Preprocessing()
sdf = SparkDataFrame()

In [4]:
cctv_paths = prepro.get_all_file_paths(PATH_CCTV_DATA)
sand_paths = prepro.get_all_file_paths(PATH_SAND_DATA)
sand_label_paths = prepro.get_all_file_paths(PATH_SAND_LABEL)

In [5]:
cctv_df = sdf.get_spark_dataframe(cctv_paths)
sand_df = sdf.get_spark_dataframe(sand_paths)
sand_label_df = sdf.get_spark_dataframe(sand_label_paths)

In [56]:
# sjson = sdf.get_spark_json(sand_label_paths[0])

In [8]:
# JSON 데이터가 압축되어 있는 경우
# df = spark.read.option("compression", "gzip").json("path/to/compressed_files/*.json")
# df.show()


In [47]:
jpg_jpeg_df = cctv_df.filter(cctv_df["file_name"].contains(".jpg") | cctv_df["file_name"].contains(".jpeg"))
png_df = cctv_df.filter(cctv_df["file_name"].contains(".png"))
etc_df = cctv_df.filter(~(cctv_df["file_name"].contains(".jpg") | cctv_df["file_name"].contains(".jpeg") | cctv_df["file_name"].contains(".png") | cctv_df["file_name"].contains(".csv")))

In [51]:
test_df = png_df.repartition(4)

In [45]:
test = cctv_df.filter(cctv_df['file_name'].contains(".csv"))

In [ ]:
test.count()

In [73]:
jpg_jpeg_df.count()

0